In [56]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

In [141]:
def f_build(d=10, n=5):
    
    a = -5.12
    b = +5.12
    
    par_A = 10.
    
    def f(I):
        X = teneva.ind_to_poi(I, a, b, n)
        y1 = par_A * d
        y2 = np.sum(X**2 - par_A * np.cos(2. * np.pi * X), axis=1)
        return y1 + y2
    
    f.d = d
    f.n = n
    
    return f

In [142]:
f = f_build()

m_trn = int(1.E+4)
I_trn = np.vstack([np.random.choice(f.n, m_trn) for k in range(f.d)]).T
y_trn = f(I_trn)

m_tst = int(1.E+4)
I_tst = np.vstack([np.random.choice(f.n, m_tst) for k in range(f.d)]).T
y_tst = f(I_tst)

In [143]:
def to_tt(Y):
    return [G.reshape(1, -1, 1) for G in Y]

In [144]:
def get(Y, i):
    return np.prod([G[j] for (G, j) in zip(Y, i)])

In [145]:
def set(Y, i, w, I_hist):
    d = len(Y)
    
    for k in range(d):
        Y[k][i[k]] *= w
             
        for i_hist in I_hist:
            if i_hist[k] != i[k]:
                continue
                    
            c = np.sum(i_hist == i)
            
            if c == 0:
                continue
            
            v = w**(d-c)

            for k2 in range(d):
                if i_hist[k2] == i[k2]:
                    continue

                Y[k2][i_hist[k2]] /= v 

In [146]:
d = f.d
n = f.n
Y = [np.ones(n) for k in range(d)]
teneva.show(to_tt(Y))

for k in range(m_trn):
    i_data = I_trn[k]
    y_data = y_trn[k]
    
    y = get(Y, i_data)
    w = (y_data / y)**(1/d)
    set(Y, i_data, w, I_trn[:k] if k>0 else [])

    e_list = []
    for q in range(k+1):
        y_pred = get(Y, I_trn[q])
        y_real = y_trn[q]
        e_list.append(np.abs(y_pred - y_real))
    e_list = ', '.join([f'{e:-7.1e}' for e in e_list])
    print(f'Iter # {k+1:-6d} : {e_list}')
  
    # print(i_data)
    # print(np.array(Y).T)
        
    if k > 10:
        break

TT-tensor    10D : |5| |5| |5| |5| |5| |5| |5| |5| |5| |5|
<rank>  =    1.0 :   \1/ \1/ \1/ \1/ \1/ \1/ \1/ \1/ \1/
Iter #      1 : 5.7e-14
Iter #      2 : 5.7e-14, 1.7e-13
Iter #      3 : 1.6e+02, 2.2e+02, 1.7e-13
Iter #      4 : 1.6e+02, 2.2e+02, 2.4e+02, 2.6e-13
Iter #      5 : 1.6e+02, 2.2e+02, 2.4e+02, 2.5e+02, 3.1e-12
Iter #      6 :     nan,     nan,     nan,     nan,     nan,     nan
Iter #      7 :     nan,     nan,     nan,     nan,     nan,     nan,     nan
Iter #      8 :     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan
Iter #      9 :     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan
Iter #     10 :     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan
Iter #     11 :     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan
Iter #     12 :     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan


/var/folders/lx/fvbjhm6j19d8w7f5ylv8x15w0000gn/T/ipykernel_83575/4232390218.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  w = (y_data / y)**(1/d)
/var/folders/lx/fvbjhm6j19d8w7f5ylv8x15w0000gn/T/ipykernel_83575/878628479.py:5: RuntimeWarning: invalid value encountered in double_scalars
  Y[k][i[k]] *= w


In [24]:
k = 0
i_data = I_trn[k]
y_data = y_trn[k]
y_data

75.60309831340474

In [28]:
y = get(Y, i_data)
w = (y_data / y)**(1/d)
set(Y, i_data, w)
get(Y, i_data)

75.60309831340477